In [1]:
from sklearn import model_selection , linear_model, metrics, pipeline,tree, preprocessing,neighbors,ensemble,svm,naive_bayes
from sklearn.model_selection import GridSearchCV,cross_val_score,train_test_split,RandomizedSearchCV,cross_val_predict
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler

#from sklearn import model_selection import grid_search
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
#from sklearn.linear_model import LogisticRegression
#from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
raw_train_data = pd.read_csv('./data/train.csv')
raw_test_data = pd.read_csv('./data/test.csv')

In [3]:
raw_train_data.shape,raw_test_data.shape,891+418
#raw_train_data_shuffled = raw_train_data.reindex(np.random.permutation(raw_train_data.index))

((891, 12), (418, 11), 1309)

In [4]:
#raw_train_data.info()

### Train

In [5]:
#Проверка пропусков
NAs = pd.concat([raw_train_data.isnull().sum()], axis=1, keys=['data'])
NAs[NAs.sum(axis=1) > 0]

,data
Age,177
Cabin,687
Embarked,2


In [6]:
def prepare_data(raw_data):
    #Обработка пропусков
    #Заполняем медианой
    #raw_data.Age = raw_data.Age.fillna(raw_data.Age.median())
    #Заполняем медианой c группировкой
    grp = raw_data.groupby(['Sex', 'Pclass'])  
    raw_data.Age = grp.Age.apply(lambda x: x.fillna(x.median()))
    #Заполняем модой
    #raw_data.Embarked = raw_data.Embarked.fillna(raw_data.Embarked.value_counts().idxmax())
    raw_data.Embarked.fillna(raw_data.Embarked.mode()[0], inplace = True)
    raw_data.Fare.fillna(raw_data.Fare.mode()[0],inplace = True)
    #Заполняем 'N'
    raw_data.Cabin = raw_data.Cabin.fillna('NA')

    #raw_data.Embarked.value_counts()
    #добавим новые признаки
    raw_data['Family'] = raw_data.Parch + raw_data.SibSp+1
    raw_data['Family_Size'] = pd.cut(raw_data.Family, [0,1.5,4.5,15],labels=['Single','SmallFamily','LargeFamily'])
    #raw_data['Is_Alone'] = (raw_data.Family == 0).astype(int)
    raw_data['Salutation'] = raw_data.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip()) 
    raw_data['Age_Range'] = pd.cut(raw_data.Age, [0, 10, 20, 30, 40, 50, 60,70,80])
    raw_data['Fare_Category'] = pd.cut(raw_data.Fare, bins=[0,7.90,14.45,31.28,120.0,513.0],include_lowest = True, labels=['Low','Mid','High_Mid','High','Upper_High'])
    raw_data['isCabin']=raw_data.Cabin.apply(lambda x: 1 if x != 'NA' else 0)
    #raw_data['Fare_Category'] = pd.cut(raw_data['Fare'], bins=[0,7.90,14.45,31.28,120.0], labels=['Low','Mid','High_Mid','High'])
    #raw_data['Fare_Category'] = pd.cut(raw_data['Fare'], [0,7.90,14.45,31.28,120.0,513.0],include_lowest = True)
    #raw_data.Salutation.nunique()
    #####################New features#####################
    name_dict = {"Capt":       "officer",
                 "Col":        "officer",
                 "Major":      "officer",
                 "Dr":         "officer",
                 "Rev":        "officer",
                 "Jonkheer":   "snob",
                 "Don":        "snob",
                 "Sir" :       "snob",
                 "the Countess":"snob",
                 "Dona":       "snob",
                 "Lady" :      "snob",
                 "Mme":        "married",
                 "Ms":         "married",
                 "Mrs" :       "married",
                 "Miss" :      "single",
                 "Mlle":       "single",
                 "Mr" :        "man",
                 "Master" :    "boy"
                }
    raw_data['Salutation_type'] = raw_data['Salutation'].map(name_dict)
    
    return 0


In [7]:
prepare_data(raw_train_data)

0

In [8]:
raw_train_data.head(4)
#display_all(df.describe(include='all').T)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Family,Family_Size,Salutation,Age_Range,Fare_Category,isCabin,Salutation_type
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NA,S,2,SmallFamily,Mr,"(20, 30]",Low,0,man
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,SmallFamily,Mrs,"(30, 40]",High,1,married
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NA,S,1,Single,Miss,"(20, 30]",Mid,0,single
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,SmallFamily,Mrs,"(30, 40]",High,1,married


In [9]:
#Проверка пропусков
NAs = pd.concat([raw_train_data.isnull().sum()], axis=1, keys=['data'])
NAs[NAs.sum(axis=1) > 0]

,data


In [10]:
#raw_train_data['Singleton']=raw_train_data['Family'].apply(lambda x: 1 if x == 1 else 0)
#raw_train_data['SmallFamily']=raw_train_data['Family'].apply(lambda x: 1 if 2<=x <= 4 else 0)
#raw_train_data['LargeFamily']=raw_train_data['Family'].apply(lambda x: 1 if 5<=x  else 0)
#Удалим исходные, которые были базой
#raw_train_data.drop(['SibSp','Parch','Age','Fare'], axis = 1, inplace = True)
#raw_train_data.isnull().any()

In [11]:
train_labels = raw_train_data['Survived']
train_data = raw_train_data.drop(['PassengerId', 'Survived'], axis = 1)

In [12]:
string_data_columns = ['Name','Ticket','Cabin']
categorical_data_columns = ['Pclass','Sex','Embarked','Salutation','Age_Range', 'Fare_Category','Family_Size','Salutation_type']
numeric_data_columns = ['Age', 'SibSp', 'Parch', 'Fare','Family','isCabin']
#numeric_data_columns = ['Family']

In [13]:
set(list(train_data.columns))==set(string_data_columns+categorical_data_columns+numeric_data_columns)

True

In [14]:
#Оставшиеся колонки
set(train_data.columns) - set(numeric_data_columns)-set(string_data_columns)-set(categorical_data_columns)

set()

In [15]:
categorical_data_indices = np.array([(column in categorical_data_columns) for column in train_data.columns], dtype = bool)
numeric_data_indices = np.array([(column in numeric_data_columns) for column in train_data.columns], dtype = bool)

### Test 

In [16]:
#Проверка пропусков
NAs = pd.concat([raw_test_data.isnull().sum()], axis=1, keys=['data'])
NAs[NAs.sum(axis=1) > 0]


,data
Age,86
Fare,1
Cabin,327


In [17]:
prepare_data(raw_test_data)

0

In [18]:
#Проверка пропусков
NAs = pd.concat([raw_test_data.isnull().sum()], axis=1, keys=['data'])
NAs[NAs.sum(axis=1) > 0]

,data


In [19]:
test_data = raw_test_data.drop(['PassengerId'], axis = 1)
#Оставшиеся колонки
set(test_data.columns) - set(numeric_data_columns)-set(string_data_columns)-set(categorical_data_columns)

set()

In [20]:
#test_data.isnull().any()

## Все колонки (лин завис)

In [21]:
#classifier = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)

In [22]:
def create_estimator(numeric_data_indices,categorical_data_indices,classifier):
    estimator = pipeline.Pipeline(steps = [       
        (
            'feature_processing', pipeline.FeatureUnion(transformer_list = [        
                #binary
                #('binary_variables_processing', preprocessing.FunctionTransformer(lambda data: data[:, binary_data_indices])), 
                        
                #numeric
                ('numeric_variables_processing', pipeline.Pipeline(steps = [
                    ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, numeric_data_indices],validate=False)),
                    ('scaling', preprocessing.StandardScaler())            
                            ])),
        
                #categorical
                ('categorical_variables_processing', pipeline.Pipeline(steps = [
                    ('selecting', preprocessing.FunctionTransformer(lambda data: data[:, categorical_data_indices],validate=False)),
                    ('hot_encoding', preprocessing.OneHotEncoder(handle_unknown = 'ignore',sparse=False))            
                            ])),
            ])
        ),
        ('model_fitting', classifier)
        ]
    )
    return estimator

In [23]:
#cross_val_score(estimator, train_data.values, train_labels, scoring = 'accuracy', cv = 3)
#estimator.fit(train_data.values, train_labels)
#train_labels = raw_train_data['Survived']

### Разделение выборки

In [24]:
X_train, X_test, y_train, y_test = train_test_split(train_data.values, train_labels, test_size=0.5,shuffle=True, random_state=42)
#stratify=train_labels

In [25]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((445, 17), (446, 17), (445,), (446,))

### оценка алгоритмов

In [26]:
# кросс валидация тратегии
cv_strategy_SKF = model_selection.StratifiedKFold(n_splits = 5, shuffle = True, random_state = 0)
cv_strategy_KF =model_selection.KFold(n_splits=5, shuffle = True, random_state = 0)
cv_strategy_ShS =model_selection.ShuffleSplit(n_splits=5, train_size=0.5, test_size=.25,random_state=0)
cv_strategy_ShS =model_selection.StratifiedShuffleSplit(n_splits=5, train_size=0.5, test_size=.25,random_state=0)
cv_dict = {'SKF':cv_strategy_SKF,'KF':cv_strategy_KF,'ShS':cv_strategy_ShS,'ShS':cv_strategy_ShS}

In [27]:
classifier_dict = {'RiC':linear_model.RidgeClassifier(),'SVC':svm.SVC(gamma='auto'),'SGD':linear_model.SGDClassifier(),'LR':linear_model.LogisticRegression(solver='lbfgs'),
                   'KNC':neighbors.KNeighborsClassifier(),'DeTree':tree.DecisionTreeClassifier(),
                   'RF':ensemble.RandomForestClassifier(n_estimators=10),'GB':ensemble.GradientBoostingClassifier(),
                   'BG':ensemble.BaggingClassifier(),'NB':naive_bayes.GaussianNB()}

In [28]:
# кросс валидация оценка

clf_res_list=[]
cv_res_list=[]
mean_res_list=[]
max_res_list=[]
min_res_list=[]
std_res_list=[]
test_acc=[]
for k, v in classifier_dict.items():
    #print("Code : {0}, Value : {1}".format(k, v))
    print('***Classifier*****',k)
    clf=create_estimator(numeric_data_indices,categorical_data_indices,v)
    for name,val in cv_dict.items():
        clf_scoring = cross_val_score(clf, X_train, y_train, scoring = 'accuracy', cv = val)
        print('--cv--',name,'----',val)
        #print ('mean:{}, max:{}, min:{}, std:{}'.format(clf_scoring.mean(), clf_scoring.max(), clf_scoring.min(), clf_scoring.std()))
        clf_res_list.append(k)
        cv_res_list.append(name)
        mean_res_list.append(clf_scoring.mean())
        max_res_list.append(clf_scoring.max())
        min_res_list.append(clf_scoring.min())
        std_res_list.append(clf_scoring.std())
        clf.fit(X_train, y_train)
        y_pred=clf.predict(X_test)
        test_acc.append(accuracy_score(y_test, y_pred))

***Classifier***** RiC
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=0.5)
***Classifier***** SVC
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=0.5)
***Classifier***** SGD
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.25,
            train_size=0.5)
***Classifier***** LR
--cv-- SKF ---- StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
--cv-- KF ---- KFold(n_splits=5, random_state=0, shuffle=True)
--cv-- ShS ---- StratifiedSh

In [29]:
pd.DataFrame({'clf':clf_res_list, 'cv':cv_res_list,'mean':mean_res_list,'max':max_res_list,'min':min_res_list,'std':std_res_list,'test_acc':test_acc})

,clf,cv,mean,max,min,std,test_acc
0,RiC,SKF,0.842706,0.866667,0.811111,0.023400,0.820628
1,RiC,KF,0.817978,0.853933,0.764045,0.031300,0.820628
2,RiC,ShS,0.816071,0.839286,0.803571,0.012111,0.820628
3,SVC,SKF,0.831343,0.866667,0.806818,0.019676,0.834081
4,SVC,KF,0.835955,0.865169,0.786517,0.028953,0.834081
5,SVC,ShS,0.819643,0.830357,0.803571,0.010412,0.834081
6,SGD,SKF,0.782203,0.818182,0.733333,0.036811,0.778027
7,SGD,KF,0.793258,0.831461,0.764045,0.022018,0.807175
8,SGD,ShS,0.760714,0.812500,0.705357,0.034069,0.786996
9,LR,SKF,0.824727,0.855556,0.800000,0.024038,0.816143


### Подбор пареметров для RidgeClassifier

In [ ]:
#SGD
#RidgeClassifier

parameters_grid = {
    'model_fitting__alpha' : [1e-3, 1e-2, 1e-1, 1,1e1,1e2], #RC
    #'model_fitting__kernel' : ['linear', 'poly', 'rbf', 'sigmoid'], #GB
    #'model_fitting__class_weight' : [None,'balanced'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 0.5, 12),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 12),
    #'model_fitting__max_features':["log2","sqrt"],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [ ]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,linear_model.RidgeClassifier(fit_intercept=False))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=40)

In [ ]:
%%time
#grid_cv.fit(train_data.values, train_labels)
grid_cv.fit(X_train, y_train)

print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
#Коэфициенты модели
#grid_cv.best_estimator_.steps[0][1].transformer_list[1][1].steps[0][1].fit_transform(X_train)[:2]
#one_hot_cat_col=grid_cv.best_estimator_.steps[0][1].transformer_list[1][1][1].get_feature_names(categorical_data_columns)
#grid_cv.best_estimator_.steps[0][1].transformer_list[1][1].steps[1][1].categories_
all_column=numeric_data_columns+list(grid_cv.best_estimator_.steps[0][1].transformer_list[1][1].steps[1][1].get_feature_names())
coef=grid_cv.best_estimator_.steps[1][1].coef_[0]
sorted(zip(np.abs(coef),all_column),reverse=True)

### Подбор пареметров для SVC

In [ ]:
#SVC

parameters_grid = {
    #'model_fitting__C' : [0.01,0.1, 1, 10, 100,], 
    'model_fitting__C' : [0.8, 0.85, 0.9],
    'model_fitting__kernel' : ['linear','rbf', 'sigmoid'], 
    'model_fitting__class_weight' : [None,'balanced'],
    'model_fitting__decision_function_shape':['ovo', 'ovr'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 0.5, 12),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 12),
    #'model_fitting__max_features':["log2","sqrt"],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [ ]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,svm.SVC(gamma='auto'))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
#clf.get_params().keys()

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=40)

In [ ]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

### Подбор пареметров для GB

In [ ]:
#GradientBoosting

parameters_grid = {
    'model_fitting__n_estimators' : [3,4, 8, 16, 32, 64, 100], #GB
    'model_fitting__max_depth' : [3,5,7], #GB
    #'model_fitting__loss' : ["deviance",'exponential'],
    'model_fitting__learning_rate': [1, 0.5, 0.25, 0.1, 0.05, 0.01],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 1, 5),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 5),
    'model_fitting__max_features':['auto',"log2","sqrt",None],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [ ]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,ensemble.GradientBoostingClassifier())
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
#clf.get_params().keys()

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=100)

In [ ]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
#y_pred=grid_cv.predict(X_test)

In [ ]:
#accuracy_score(y_test, y_pred)

### Подбор пареметров для LR

In [ ]:
#LR

parameters_grid = {
    'model_fitting__C' : [0.001,0.01,0.1,1,10,100], #GB
    #'model_fitting__penalty' : ['l2','l1','none'],
    'model_fitting__solver' : ['newton-cg', 'lbfgs', 'liblinear']
    
}

In [ ]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,linear_model.LogisticRegression(max_iter=1000))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=100)

In [ ]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

### Подбор пареметров для RF

In [ ]:
#RandomForestClassifier(n_estimators=10)

parameters_grid = {
    'model_fitting__n_estimators' : [1, 2, 4, 8, 16, 32, 64, 100, 200,500], #GB
    #'model_fitting__max_depth' : np.linspace(1, 32, 32, endpoint=True), #GB
    #'model_fitting__loss' : ["deviance",'exponential'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    'model_fitting__min_samples_split': [2,5,7,10],
    'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True),
    #'model_fitting__max_features':["log2","sqrt",'auto'],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
    #'model_fitting__class_weight' : ['balanced',None]
}

In [ ]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,ensemble.RandomForestClassifier(class_weight='balanced'))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=100)

In [ ]:
%%time
grid_cv.fit(X_train, y_train)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
y_pred=grid_cv.predict(X_test)
accuracy_score(y_test, y_pred)

## Обучение финального классификатора ALL_TRAIN

In [ ]:
#SVC

parameters_grid = {
    'model_fitting__gamma' : ['auto',0.0001,0.001,0.01,0.1,1,10], 
    'model_fitting__C' : [0.01,0.1, 1, 10],
    'model_fitting__kernel' : ['linear','rbf', 'sigmoid'], #GB
    'model_fitting__class_weight' : [None,'balanced'],
    'model_fitting__decision_function_shape':['ovo', 'ovr'],
    #'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 0.5, 12),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 12),
    #'model_fitting__max_features':["log2","sqrt"],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [ ]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,svm.SVC(gamma='auto'))
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=40)

In [ ]:
%%time
grid_cv.fit(train_data.values, train_labels)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

### формирование файла Kaggle

In [ ]:
result = grid_cv.predict(test_data.values)
submission = pd.DataFrame({'PassengerId':raw_test_data.PassengerId,'Survived':result})
submission.Survived = submission.Survived.astype(int)
print(submission.shape)


In [ ]:
filename = 'titanic13_SVC.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

In [ ]:
submission

In [ ]:
result

In [ ]:
#Преобразование категориальных признаков в отдельном df
dummy_data=pd.get_dummies(data=raw_test_data, columns=categorical_data_columns)
dummy_index=dummy_data.columns.str.contains('|'.join(categorical_data_columns))
dummy_data=dummy_data.loc[:,dummy_index]

In [ ]:
#Числовые данные в отдельном df
numeric_data=raw_test_data[numeric_data_columns]

In [ ]:
#создаем стандартный scaler
#scaler = StandardScaler()
scaled_test_data=scaler.transform(numeric_data)
#scaled_test_data = scaler.transform(test_data)

In [ ]:
all_test_data=np.hstack((scaled_test_data,dummy_data.values))

In [ ]:
pd.concat([numeric_data, dummy_data], axis=1)

In [ ]:
dummy_data.head()

In [ ]:
numeric_data.head()

In [ ]:
#raw_train_data.Survived.value_counts()

In [ ]:
train_labels = raw_train_data['Survived']
train_data = raw_train_data.drop(['PassengerId', 'Survived'], axis = 1)

In [ ]:
#train_data.info()

In [ ]:
#train_data['Sex'].fillna(value=0,inplace=True)
train_data['Embarked'].fillna(value='0',inplace=True)
train_data['Age'].fillna(value=0,inplace=True)

In [ ]:
# найти все признаки, в которых первое значение - строка
def find_cat(data):
    for name in data.columns:
        s = ''
        s += name
        if (type(data[name][0]) == str):
            s += ' строка,'
        if (data[name].nunique()<=3):
            s += ' мало уникальных'
        if (s!=name):
            print (s)
            
find_cat(train_data)

In [ ]:
string_data_columns = ['Name','Ticket','Cabin']

In [ ]:
categorical_data_columns = ['Pclass','Sex','Embarked'] 
categorical_data_indices = np.array([(column in categorical_data_columns) for column in train_data.columns], dtype = bool)

In [ ]:
categorical_data_indices

In [ ]:
numeric_data_columns = ['Age', 'SibSp', 'Parch', 'Fare']
numeric_data_indices = np.array([(column in numeric_data_columns) for column in train_data.columns], dtype = bool)

In [ ]:
numeric_data_indices

In [ ]:
#Оставшиеся колонки
set(train_data.columns) - set(numeric_data_columns)-set(string_data_columns)-set(categorical_data_columns)

In [ ]:
train_data.isnull().any()

In [ ]:
pd.DataFrame({'clf':clf_res_list, 'cv':cv_res_list,'mean':mean_res_list,'max':max_res_list,'min':min_res_list,'std':std_res_list})

### Подбор пареметров для GB

In [ ]:
#GradientBoosting

parameters_grid = {
    'model_fitting__n_estimators' : [100,250,500], #GB
    'model_fitting__max_depth' : [3,5], #GB
    #'model_fitting__loss' : ["deviance",'exponential'],
    'model_fitting__learning_rate': [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    #'model_fitting__learning_rate': [0.2],
    #'model_fitting__min_samples_split': np.linspace(0.1, 0.5, 12),
    #'model_fitting__min_samples_leaf': np.linspace(0.1, 0.5, 12),
    #'model_fitting__max_features':["log2","sqrt"],
    #'model_fitting__criterion': ["friedman_mse",  "mae"],
    #"model_fitting__subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    #'model_fitting__subsample':[0.85]
}

In [ ]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,ensemble.GradientBoostingClassifier())
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
#clf.get_params()

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=40)

In [ ]:
%%time
grid_cv.fit(train_data.values, train_labels)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
#LR

parameters_grid = {
    'model_fitting__C' : [0.001,0.01,0.1,1,10,100], #GB
    
}

In [ ]:
clf=create_estimator(numeric_data_indices,categorical_data_indices,linear_model.LogisticRegression())
cv = model_selection.StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0)

In [ ]:
grid_cv = GridSearchCV(clf, parameters_grid, cv = cv)

In [ ]:
grid_cv = RandomizedSearchCV(clf, parameters_grid, cv = cv,n_iter=40)

In [ ]:
%%time
grid_cv.fit(train_data.values, train_labels)
print (grid_cv.best_score_)
print (grid_cv.best_params_)

### Стекинг

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.33, random_state=42)

In [ ]:
clfs = [ensemble.RandomForestClassifier(n_estimators=250, n_jobs=-1),
        linear_model.LogisticRegression(class_weight=None,tol=1e-05, n_jobs=-1, penalty='l2'),
        ensemble.GradientBoostingClassifier(n_estimators=250),svm.SVC()]

In [ ]:
clfs = [svm.SVC(),
        linear_model.LogisticRegression(class_weight=None,tol=1e-05, n_jobs=-1, penalty='l2')]

In [ ]:
dataset_blend_train = np.zeros((train_data.shape[0], len(clfs)))

In [ ]:
kf = model_selection.StratifiedKFold(n_splits=5,shuffle=False)
for j, clf in enumerate(clfs):
    print (j, clf)
    for i, (train_, test_) in enumerate(kf.split(train_data, train_labels)):
        print ("Fold", i)
        X_tr = train_data.values[train_]
        y_tr = train_labels[train_]
        X_te = train_data.values[test_]
        y_te = train_labels[test_]
        clf_pipeline=create_estimator(numeric_data_indices,categorical_data_indices,clf)
        clf_pipeline.fit(X_tr, y_tr)
        print('------------------------',type(clf))
        if isinstance(clf,svm._classes.SVC):
            print('----------SVM-------')
            y_submission = clf_pipeline.predict(X_te)
        else:
            print('----------Other-------')
            y_submission = clf_pipeline.predict_proba(X_te)[:, 1]
        dataset_blend_train[test_, j] = y_submission

In [ ]:
clf2 = linear_model.LogisticRegression(C=0.01,class_weight=None,tol=1e-05, random_state=11, n_jobs=-1)
ls = cross_val_score(clf2, dataset_blend_train, train_labels, cv=5, scoring='accuracy')
print(ls)
np.mean(ls)

In [ ]:
list(kf.split(train_data, train_labels))[4]

In [ ]:
#Без обработки признаков для деревьев

In [ ]:
train_data[categorical_data_columns+numeric_data_columns].values

In [ ]:
#classifier_dict = {'RF':ensemble.RandomForestClassifier()}
#cross_val_score(ensemble.RandomForestClassifier(), train_data[categorical_data_columns+numeric_data_columns].values, train_labels, scoring = 'accuracy', cv = 5)

In [ ]:
clf

In [ ]:
mean_list

In [ ]:
# кросс валидация оценка
for cv in cv_list:
    #-----------------
    
    #-----------------
    clf_scoring = cross_val_score(clf, train_data.values, train_labels, scoring = 'accuracy', cv = cv)
    print('--cv--',cv)
    print ('mean:{}, max:{}, min:{}, std:{}'.format(clf_scoring.mean(), clf_scoring.max(), 
                                                     clf_scoring.min(), clf_scoring.std()))

In [ ]:
#estimator.fit(train_data.values, train_labels.values)

In [ ]:
estimator.get_params()

In [ ]:
parameters_grid = {
    'model_fitting__max_iter' : [1000],
#    'model_fitting__eta0' : [0.001, 0.05],
}

In [ ]:
grid_cv = GridSearchCV(estimator, parameters_grid, cv = 4)

In [ ]:
%%time
grid_cv.fit(train_data.values, train_labels)

In [ ]:
print (grid_cv.best_score_)
print (grid_cv.best_params_)

In [ ]:
#pd.get_dummies(X_train_cat, prefix=['col1', 'col2'])

In [ ]:
#from sklearn.preprocessing import OneHotEncoder
#ohe = OneHotEncoder(sparse=False) # sparse=False categorical_features=...
#new_ohe_features = ohe.fit_transform(X_train_cat.city_le.values.reshape(-1, 1))

In [ ]:
X_train_cat

In [ ]:
plt.hist(raw_train_data.Pclass.values,bins=3)

In [ ]:
categ_cols = 